In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, BatchNorm, global_add_pool, ChebConv, global_max_pool, SAGPooling, GATConv, GATv2Conv, TransformerConv, SuperGATConv, global_mean_pool, Linear
from torch.nn import BatchNorm1d
from math import floor
import torch
import torch_geometric
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from pathlib import Path

In [34]:
class MultiLevelConvNet(nn.Module):
    """Same as EEGGraphConvNet but with fewer 
    convolutional layers
    """
    def __init__(self, **kwargs):
        super(MultiLevelConvNet, self).__init__()
        # Layers definition
        # Graph convolutional layers
        self.conv1 = GCNConv(153550, 32, cached=True, normalize=False)
        self.conv2 = GCNConv(32, 32, cached=True, normalize=False)
        self.conv3 = GCNConv(32, 64, cached=True, normalize=False)
        
        
        # Batch normalization
        self.batch_norm1 = BatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm2 = BatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm3 = BatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        
        self.fc1 = nn.Linear(32, 64)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 64)
        
        # Fully connected layers
        self.classifier = nn.Sequential(
            nn.Linear(192, 64),
            nn.LeakyReLU(),
            nn.Linear(64, 32),
            nn.LeakyReLU(),
            nn.Linear(32, 2),
        )
        
        # Xavier initializacion for fully connected layers
        self.fc1.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc2.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc3.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        
        
    def forward(self, x, edge_index, edge_weigth, batch):
        x1 = F.leaky_relu(self.batch_norm1(self.conv1(x, edge_index, edge_weigth)), negative_slope=0.01)
        x2 = F.leaky_relu(self.batch_norm2(self.conv2(x1, edge_index, edge_weigth)), negative_slope=0.01)
        x3 = F.leaky_relu(self.batch_norm3(self.conv3(x2, edge_index, edge_weigth)), negative_slope=0.01)
        
        add_pool1 = global_add_pool(x1, batch=batch)
        add_pool2 = global_add_pool(x2, batch=batch)
        add_pool3 = global_add_pool(x3, batch=batch)
        
        out1 = F.leaky_relu(self.fc1(add_pool1), negative_slope=0.01)        
        out2 = F.leaky_relu(self.fc2(add_pool2), negative_slope=0.01)        
        out3 = F.leaky_relu(self.fc3(add_pool3), negative_slope=0.01)
        
        out = torch.cat((out1, out2, out3), dim=1)        
        out = self.classifier(out)
        return out

In [11]:
def create_corrected_data_list(path):
    data_list = list()
    for file in path.iterdir():
        data_list.append(torch.load(file))
    corrected_data_list = list()
    for data in data_list:
    # print(data)
        data = torch_geometric.data.Data(
            x=torch.tensor(data.x),
            edge_index=torch.tensor(data.edge_index),
            edge_attr=torch.tensor(data.edge_attr),
            label=torch.tensor(data.label),
        )
        corrected_data_list.append(data)
    
    rm = [
      7,
      14+1,
      14+2,
      17+3,
      17+4,
      26+5,
      38+6,
      54+7,
      65+8,
      69+9
      ]

    dl = list()
    start = 0
    for r in rm:
        dl.extend(corrected_data_list[start:r])
        start = r + 1

    dl.extend(corrected_data_list[start:])
    dl_filterd = list()
    for data in dl:
        if data.label == 2:
            # print(data.label)
            # if data.label == 2:
            data.label = torch.tensor(1)
        dl_filterd.append(data)

    len(dl_filterd)
    return dl_filterd
            
    

## Moments Pearson

In [12]:
path = Path('graphs/moments_pearson/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_54096\598882657.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [13]:
model = MultiLevelConvNet()

In [7]:
dl_filterd

[Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=[19, 19], label=1),
 Data(x=[19, 6], edge_index=[2, 361], edge_attr=

In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 26530


MultiLevelConvNet(
  (conv1): GCNConv(6, 32)
  (conv2): GCNConv(32, 32)
  (conv3): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(32)
  (batch_norm3): BatchNorm(64)
  (fc1): Linear(in_features=32, out_features=64, bias=True)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (classifier): Sequential(
    (0): Linear(in_features=192, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
)

In [15]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=4.607035	 acc=0.538462
Test  : epoch=1	 loss=275.223943	 acc=0.500000

Train : epoch=2	 loss=5.213089	 acc=0.589744
Test  : epoch=2	 loss=1936.248680	 acc=0.500000

Train : epoch=3	 loss=4.765303	 acc=0.615385
Test  : epoch=3	 loss=58.456558	 acc=0.562500

Train : epoch=4	 loss=5.065197	 acc=0.653846
Test  : epoch=4	 loss=140.867846	 acc=0.500000

Train : epoch=5	 loss=4.698177	 acc=0.653846
Test  : epoch=5	 loss=165.044997	 acc=0.500000

Train : epoch=6	 loss=3.822331	 acc=0.653846
Test  : epoch=6	 loss=163.609787	 acc=0.500000

Train : epoch=7	 loss=3.844903	 acc=0.653846
Test  : epoch=7	 loss=171.055600	 acc=0.500000

Train : epoch=8	 loss=3.731880	 acc=0.653846
Test  : epoch=8	 loss=191.620355	 acc=0.500000

Train : epoch=9	 loss=3.710398	 acc=0.653846
Test  : epoch=9	 loss=179.527705	 acc=0.500000

Train : epoch=10	 loss=3.742423	 acc=0.653846
Test  : epoch=10	 loss=205.975324	 acc=0.500000



In [16]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## Moments PLI

In [17]:
path = Path('graphs/moments_pli/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_54096\598882657.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [18]:
model = MultiLevelConvNet()

In [19]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 26530


MultiLevelConvNet(
  (conv1): GCNConv(6, 32)
  (conv2): GCNConv(32, 32)
  (conv3): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(32)
  (batch_norm3): BatchNorm(64)
  (fc1): Linear(in_features=32, out_features=64, bias=True)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (classifier): Sequential(
    (0): Linear(in_features=192, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
)

In [20]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=4.567647	 acc=0.576923
Test  : epoch=1	 loss=17304.725346	 acc=0.500000

Train : epoch=2	 loss=5.922574	 acc=0.653846
Test  : epoch=2	 loss=22383.266221	 acc=0.500000

Train : epoch=3	 loss=4.864628	 acc=0.653846
Test  : epoch=3	 loss=29024.502947	 acc=0.500000

Train : epoch=4	 loss=4.939911	 acc=0.653846
Test  : epoch=4	 loss=35460.005848	 acc=0.500000

Train : epoch=5	 loss=4.351013	 acc=0.653846
Test  : epoch=5	 loss=24331.076101	 acc=0.500000

Train : epoch=6	 loss=4.911747	 acc=0.653846
Test  : epoch=6	 loss=32108.777561	 acc=0.500000

Train : epoch=7	 loss=4.006101	 acc=0.653846
Test  : epoch=7	 loss=18047.943954	 acc=0.500000

Train : epoch=8	 loss=3.997586	 acc=0.653846
Test  : epoch=8	 loss=25389.779441	 acc=0.500000

Train : epoch=9	 loss=3.748299	 acc=0.653846
Test  : epoch=9	 loss=23401.344031	 acc=0.500000

Train : epoch=10	 loss=3.870325	 acc=0.653846
Test  : epoch=10	 loss=27075.311075	 acc=0.500000



In [21]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## PSD CSD

In [22]:
path = Path('graphs/psd_csd/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_54096\598882657.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [23]:
model = MultiLevelConvNet()

In [24]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 26530


MultiLevelConvNet(
  (conv1): GCNConv(6, 32)
  (conv2): GCNConv(32, 32)
  (conv3): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(32)
  (batch_norm3): BatchNorm(64)
  (fc1): Linear(in_features=32, out_features=64, bias=True)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (classifier): Sequential(
    (0): Linear(in_features=192, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
)

In [25]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=2.269136	 acc=0.653846
Test  : epoch=1	 loss=1.690443	 acc=0.500000

Train : epoch=2	 loss=3.965007	 acc=0.653846
Test  : epoch=2	 loss=2.507696	 acc=0.500000

Train : epoch=3	 loss=3.955566	 acc=0.653846
Test  : epoch=3	 loss=2.515073	 acc=0.500000

Train : epoch=4	 loss=3.948190	 acc=0.653846
Test  : epoch=4	 loss=2.658419	 acc=0.500000

Train : epoch=5	 loss=3.963414	 acc=0.653846
Test  : epoch=5	 loss=2.900081	 acc=0.500000

Train : epoch=6	 loss=4.200440	 acc=0.653846
Test  : epoch=6	 loss=2.623206	 acc=0.500000

Train : epoch=7	 loss=3.965634	 acc=0.653846
Test  : epoch=7	 loss=2.626941	 acc=0.500000

Train : epoch=8	 loss=4.032350	 acc=0.653846
Test  : epoch=8	 loss=3.397943	 acc=0.500000

Train : epoch=9	 loss=3.928791	 acc=0.653846
Test  : epoch=9	 loss=2.600218	 acc=0.500000

Train : epoch=10	 loss=3.845010	 acc=0.653846
Test  : epoch=10	 loss=2.880694	 acc=0.500000



In [26]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## PSD Pearson

In [27]:
path = Path('graphs/psd_pearson/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_54096\598882657.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [28]:
model = MultiLevelConvNet()

In [29]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 26530


MultiLevelConvNet(
  (conv1): GCNConv(6, 32)
  (conv2): GCNConv(32, 32)
  (conv3): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(32)
  (batch_norm3): BatchNorm(64)
  (fc1): Linear(in_features=32, out_features=64, bias=True)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (classifier): Sequential(
    (0): Linear(in_features=192, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
)

In [30]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=4.485378	 acc=0.538462
Test  : epoch=1	 loss=44.693730	 acc=0.500000

Train : epoch=2	 loss=5.232894	 acc=0.615385
Test  : epoch=2	 loss=390.310431	 acc=0.500000

Train : epoch=3	 loss=4.602283	 acc=0.589744
Test  : epoch=3	 loss=199.319306	 acc=0.500000

Train : epoch=4	 loss=4.884346	 acc=0.564103
Test  : epoch=4	 loss=78.043191	 acc=0.500000

Train : epoch=5	 loss=5.267218	 acc=0.653846
Test  : epoch=5	 loss=128.112685	 acc=0.500000

Train : epoch=6	 loss=4.420438	 acc=0.653846
Test  : epoch=6	 loss=191.420033	 acc=0.500000

Train : epoch=7	 loss=3.932077	 acc=0.653846
Test  : epoch=7	 loss=195.563075	 acc=0.500000

Train : epoch=8	 loss=3.973272	 acc=0.653846
Test  : epoch=8	 loss=226.879498	 acc=0.500000

Train : epoch=9	 loss=3.819469	 acc=0.653846
Test  : epoch=9	 loss=221.417345	 acc=0.500000

Train : epoch=10	 loss=3.763641	 acc=0.653846
Test  : epoch=10	 loss=262.901756	 acc=0.500000



In [31]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## RAW Pearson

In [32]:
path = Path('graphs/raw_pearson/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_54096\598882657.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(data.x),
C:\Users\mashh\AppData\Local\Temp\ipykernel_54096\598882657.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [33]:
train_dl[0]

Data(x=[19, 153550], edge_index=[2, 361], edge_attr=[19, 19], label=0)

In [35]:
model = MultiLevelConvNet()

In [36]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 4939938


MultiLevelConvNet(
  (conv1): GCNConv(153550, 32)
  (conv2): GCNConv(32, 32)
  (conv3): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(32)
  (batch_norm3): BatchNorm(64)
  (fc1): Linear(in_features=32, out_features=64, bias=True)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (classifier): Sequential(
    (0): Linear(in_features=192, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
)

In [37]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=5.411580	 acc=0.512821
Test  : epoch=1	 loss=6.123243	 acc=0.562500

Train : epoch=2	 loss=4.787309	 acc=0.602564
Test  : epoch=2	 loss=177.158327	 acc=0.500000

Train : epoch=3	 loss=6.035715	 acc=0.525641
Test  : epoch=3	 loss=45.219517	 acc=0.500000

Train : epoch=4	 loss=5.331803	 acc=0.564103
Test  : epoch=4	 loss=69.755356	 acc=0.500000

Train : epoch=5	 loss=5.509113	 acc=0.525641
Test  : epoch=5	 loss=33.589036	 acc=0.500000

Train : epoch=6	 loss=4.216980	 acc=0.653846
Test  : epoch=6	 loss=61.875588	 acc=0.500000

Train : epoch=7	 loss=3.680024	 acc=0.653846
Test  : epoch=7	 loss=120.918018	 acc=0.500000

Train : epoch=8	 loss=3.960026	 acc=0.653846
Test  : epoch=8	 loss=172.133124	 acc=0.500000

Train : epoch=9	 loss=3.687382	 acc=0.653846
Test  : epoch=9	 loss=209.716006	 acc=0.500000

Train : epoch=10	 loss=3.711567	 acc=0.653846
Test  : epoch=10	 loss=229.410045	 acc=0.500000



In [38]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## Raw PLI

In [39]:
path = Path('graphs/raw_pli/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_54096\598882657.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(data.x),
C:\Users\mashh\AppData\Local\Temp\ipykernel_54096\598882657.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [40]:
model = MultiLevelConvNet()

In [41]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 4939938


MultiLevelConvNet(
  (conv1): GCNConv(153550, 32)
  (conv2): GCNConv(32, 32)
  (conv3): GCNConv(32, 64)
  (batch_norm1): BatchNorm(32)
  (batch_norm2): BatchNorm(32)
  (batch_norm3): BatchNorm(64)
  (fc1): Linear(in_features=32, out_features=64, bias=True)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (classifier): Sequential(
    (0): Linear(in_features=192, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
)

In [42]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=3.607204	 acc=0.615385
Test  : epoch=1	 loss=3875.349578	 acc=0.500000

Train : epoch=2	 loss=4.480674	 acc=0.653846
Test  : epoch=2	 loss=6244.634729	 acc=0.500000

Train : epoch=3	 loss=4.220679	 acc=0.653846
Test  : epoch=3	 loss=7125.972633	 acc=0.500000

Train : epoch=4	 loss=4.331254	 acc=0.653846
Test  : epoch=4	 loss=7132.328710	 acc=0.500000

Train : epoch=5	 loss=4.103951	 acc=0.653846
Test  : epoch=5	 loss=5991.051929	 acc=0.500000

Train : epoch=6	 loss=4.000556	 acc=0.653846
Test  : epoch=6	 loss=6014.055830	 acc=0.500000

Train : epoch=7	 loss=4.272844	 acc=0.653846
Test  : epoch=7	 loss=5923.714538	 acc=0.500000

Train : epoch=8	 loss=3.896520	 acc=0.653846
Test  : epoch=8	 loss=4552.117884	 acc=0.500000



In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))
